In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/xllm-test-v1/output (1).csv
/kaggle/input/xllm-test-v1/results.csv


In [2]:
df = pd.read_csv('/kaggle/input/xllm-test-v1/results.csv')

In [3]:
df.sample(20)

,File Name,Transcription,predicted_event
86,test-431.wav,"addressing palestinian complaints, and israel...",NaN
477,test-339.wav,"the houston energy company, which has been th...",Declare-Bankruptcy bankruptcy Declare-Bankrupt...
458,test-524.wav,"in washington joint chiefs of staff, chairman...",NaN
561,test-50.wav,it was believed nocia was a practicing sorcer...,Attack eliminate Attack eliminate Attacker noc...
206,test-395.wav,"the woman's parents, william and nancy scots,...",Die body of Victim baby Agent parents Place house
662,test-606.wav,"while at fox, he was instrumental in creating...",NaN
432,test-165.wav,rodon called for a fair and equal footing in ...,Meet talks Entity rodon
663,test-438.wav,"with opposition hardening, the white house le...",Attack war
2,test-137.wav,"uh, christian right-tack of this campaigning ...",Attack persecutor christians Attack persecutor...
94,test-179.wav,chief executive andrew harris said the compan...,Transfer-Ownership acquire Transfer-Ownership ...


In [4]:
df.loc[419]['predicted_event']

'Sue subpoena Subpo'

In [5]:
event_schema = ["Injure", "Charge-Indict", "Execute", "Merge-Org", "Start-Position", "Die", "Acquit", "Extradite", "Pardon", "Convict", "Phone-Write", "End-Org", "Release-Parole", "Transfer-Money", "Marry", "Appeal", "Fine", "Be-Born", "Transport", "Sentence", "Declare-Bankruptcy", "Divorce", "Meet", "Demonstrate", "Attack", "Start-Org", "Arrest-Jail", "Elect", "End-Position", "Sue", "Trial-Hearing", "Nominate", "Transfer-Ownership"]

In [6]:
from tqdm import tqdm
results = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating"):

    temp = {
        'id': row['File Name'].split('.')[0],
        'event': []
    }
    
    if pd.isna(row['predicted_event']):
        results.append(temp)
        continue

    tokens = row['predicted_event'].split()
    temp2 = []
    i = 0

    while i < len(tokens):
        token = tokens[i]
        if token in event_schema and (i + 1 < len(tokens)):
            if len(temp['event']) > 0:
                temp['event'][-1]['arguments'] = temp2
            temp2 = []
            temp['event'].append({
                'trigger': tokens[i + 1],
                'type': token,
                'arguments': []
            })
            i += 2
        elif token[0].isupper():
            role = token
            name_tokens = []
            i += 1
            while i < len(tokens) and tokens[i][0].islower():
                name_tokens.append(tokens[i])
                i += 1
            if name_tokens:
                temp2.append({
                    'role': role,
                    'name': ' '.join(name_tokens)
                })
        else:
            i += 1
    if len(temp['event']) > 0:
        temp['event'][-1]['arguments'] = temp2

    results.append(temp)


Generating: 100%|██████████| 676/676 [00:00<00:00, 19251.42it/s]


In [7]:
results[:10]

[{'id': 'test-49', 'event': []},
 {'id': 'test-508', 'event': []},
 {'id': 'test-137',
  'event': [{'trigger': 'persecutor', 'type': 'Attack', 'arguments': []},
   {'trigger': 'persecutor', 'type': 'Attack', 'arguments': []}]},
 {'id': 'test-286',
  'event': [{'trigger': 'forming',
    'type': 'Start-Org',
    'arguments': [{'role': 'Org', 'name': 'government'},
     {'role': 'Place', 'name': 'iraq'}]}]},
 {'id': 'test-329',
  'event': [{'trigger': 'telephone', 'type': 'Phone-Write', 'arguments': []},
   {'trigger': 'telephone', 'type': 'Phone-Write', 'arguments': []},
   {'trigger': 'Phone-Write', 'type': 'Phone-Write', 'arguments': []},
   {'trigger': 'telephone', 'type': 'Phone-Write', 'arguments': []}]},
 {'id': 'test-60',
  'event': [{'trigger': 'meeting',
    'type': 'Meet',
    'arguments': [{'role': 'Place', 'name': 'beijing'},
     {'role': 'Entity', 'name': 'washington'},
     {'role': 'Entity', 'name': 'pyongyang'}]}]},
 {'id': 'test-14',
  'event': [{'trigger': 'war', 'type

In [8]:
len(results)

676

In [9]:
results_sorted = sorted(results, key=lambda x: int(x['id'].split('-')[-1]))

In [10]:
import json

with open("results.json", "w", encoding="utf-8") as f:
    json.dump(results_sorted, f, ensure_ascii=False, indent=4)